In [2]:
from nba_api.stats.endpoints import leaguedashteamstats, teamgamelogs, leaguegamefinder
from nba_api.stats.static import teams
from datetime import date
import requests
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
cavs = 1610612739
clippers = 1610612746
knicks = 1610612752
celtics = 1610612738
hawks = 1610612737
nets = 1610612751
hornets = 1610612766
bulls = 1610612741
mavericks = 1610612742
nuggets = 1610612743
pistons = 1610612765
warriors = 1610612744
rockets = 1610612745
pacers = 1610612754
lakers = 1610612747
grizzlies = 1610612763
heat = 1610612748
bucks = 1610612749
timberwolves = 1610612750
pelicans = 1610612740
thunder = 1610612760
magic = 1610612753
sixers = 1610612755
suns = 1610612756
blazers = 1610612757
kings = 1610612758
spurs = 1610612759
raptors = 1610612761
jazz = 1610612762
wizards = 1610612764

league_ids = [cavs, clippers, knicks, celtics, hawks, nets, hornets, bulls, mavericks, nuggets, pistons, warriors, rockets,
             pacers, lakers, grizzlies, heat, bucks, timberwolves, pelicans, thunder, magic, sixers, suns, blazers, kings,
             spurs, raptors, jazz, wizards]
no_cavs = [clippers, knicks, celtics, hawks, nets, hornets, bulls, mavericks, nuggets, pistons, warriors, rockets,
          pacers, lakers, grizzlies, heat, bucks, timberwolves, pelicans, thunder, magic, sixers, suns, blazers, kings,
          spurs, raptors, jazz, wizards]

In [4]:
def combine_team_games(df, keep_method='home'):
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_AWAY', '_HOME'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_AWAY != joined.TEAM_ID_HOME]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_AWAY.str.contains(' @ ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_AWAY.str.contains(' vs. ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_AWAY == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_AWAY == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
#     display(joined)
    return result

In [5]:
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]

def get_dfs(team_id):
    seasons = ['2022-23', '2021-22', '2020-21', '2019-20']
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable='2023-24')
    all_team_games = gamefinder.get_data_frames()[0]
    
    for i in seasons:
        new_gf = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=i)
        new_games = new_gf.get_data_frames()[0]
        all_team_games = all_team_games.append(new_games)

    all_team_games.index = range(0, all_team_games.shape[0])
    game_ids = []
    for i in range(all_team_games.shape[0]):
        game_ids.append(all_team_games.loc[i]['GAME_ID'])
        
    rows = []
    for id in game_ids:
        get_games = all_games[all_games.GAME_ID == id]
        new_row = combine_team_games(get_games).iloc[0]
        rows.append(new_row)
    df = pd.DataFrame(rows)
    df_away = df[df['TEAM_ID_AWAY'].isin([team_id])]
    df_home = df[df['TEAM_ID_HOME'].isin([team_id])]
    df_home = df_home.sort_values(by='GAME_DATE', ascending=True)
    df_away = df_away.sort_values(by='GAME_DATE', ascending=True)
    df_away.index = range(0,df_away.shape[0])
    df_home.index = range(0,df_home.shape[0])
    return df_home, df_away

In [6]:
df_home, df_away = get_dfs(mavericks)
pd.set_option('max_columns', None)
df_away

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME
0,12019,1610612742,DAL,Dallas Mavericks,0011900021,2019-10-08,DAL @ OKC,L,241,104,34,94,0.362,13,50,0.260,23,31,0.742,14,31,45,22,10,6,12,28,-15.0,1610612760,OKC,Oklahoma City Thunder,OKC vs. DAL,W,238,119,41,77,0.532,14,28,0.500,23,35,0.657,8,42,50,24,4,2,19,26,15.0
1,12019,1610612742,DAL,Dallas Mavericks,0011900024,2019-10-09,DAL @ DET,L,239,117,37,79,0.468,13,35,0.371,30,38,0.789,5,27,32,22,9,3,14,23,-7.0,1610612765,DET,Detroit Pistons,DET vs. DAL,W,242,124,46,89,0.517,18,45,0.400,14,20,0.700,12,31,43,31,6,5,19,21,7.0
2,12019,1610612742,DAL,Dallas Mavericks,0011900073,2019-10-17,DAL @ LAC,W,240,102,35,78,0.449,15,39,0.385,17,20,0.850,9,49,58,25,1,4,30,23,15.0,1610612746,LAC,LA Clippers,LAC vs. DAL,L,239,87,34,93,0.366,8,33,0.242,11,18,0.611,7,32,39,21,15,1,15,26,-15.0
3,22019,1610612742,DAL,Dallas Mavericks,0021900021,2019-10-25,DAL @ NOP,W,241,123,45,93,0.484,14,40,0.350,19,26,0.731,12,37,49,23,8,7,11,19,7.0,1610612740,NOP,New Orleans Pelicans,NOP vs. DAL,L,239,116,45,98,0.459,15,41,0.366,11,15,0.733,12,36,48,30,6,3,14,22,-7.0
4,22019,1610612742,DAL,Dallas Mavericks,0021900053,2019-10-29,DAL @ DEN,W,239,109,41,84,0.488,13,37,0.351,14,24,0.583,10,32,42,29,6,5,8,14,3.0,1610612743,DEN,Denver Nuggets,DEN vs. DAL,L,240,106,43,91,0.473,11,32,0.344,9,13,0.692,14,36,50,24,6,1,10,24,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,22023,1610612742,DAL,Dallas Mavericks,0022301144,2024-04-09,DAL @ CHA,W,240,130,50,93,0.538,17,50,0.340,13,20,0.650,16,39,55,35,6,7,14,19,26.0,1610612766,CHA,Charlotte Hornets,CHA vs. DAL,L,240,104,39,89,0.438,10,32,0.313,16,21,0.762,8,31,39,24,8,3,11,16,-26.0
224,22023,1610612742,DAL,Dallas Mavericks,0022301161,2024-04-10,DAL @ MIA,W,241,111,43,87,0.494,15,44,0.341,10,13,0.769,8,31,39,27,8,8,7,13,19.0,1610612748,MIA,Miami Heat,MIA vs. DAL,L,239,92,33,81,0.407,13,30,0.433,13,13,1.000,12,34,46,22,3,1,15,16,-19.0
225,22023,1610612742,DAL,Dallas Mavericks,0022301196,2024-04-14,DAL @ OKC,L,241,86,32,97,0.330,8,39,0.205,14,17,0.824,12,29,41,19,8,2,14,15,-49.0,1610612760,OKC,Oklahoma City Thunder,OKC vs. DAL,W,241,135,54,97,0.557,14,34,0.412,13,16,0.813,11,47,58,38,6,6,14,17,49.0
226,42023,1610612742,DAL,Dallas Mavericks,0042300171,2024-04-21,DAL @ LAC,L,241,97,31,80,0.388,10,33,0.303,25,33,0.758,9,32,41,14,7,9,8,15,-12.0,1610612746,LAC,LA Clippers,LAC vs. DAL,W,240,109,40,87,0.460,18,36,0.500,11,13,0.846,10,35,45,21,5,3,12,22,12.0


In [7]:
def rolling_averages(group, cols, new_cols):
    rolling_stats = group[cols].rolling(5, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [8]:
cols = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY', 'PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']
away_cols = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY']
home_cols = ['PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']
preds = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY', 'PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']

new_cols = [f"{c}_rolling" for c in cols]
new_cols_home = [f"{c}_rolling" for c in home_cols]
new_cols_home.append(home_cols[-1])
new_cols_away = [f"{c}_rolling" for c in away_cols]
new_cols_away.append(away_cols[-1])

pred_cols = new_cols_away+new_cols_home

preds = [f"{c}_rolling" for c in preds]

def add_new_cols(team_id):
    df_home, df_away = get_dfs(team_id)
    
    df_home = rolling_averages(df_home, cols, new_cols)
    df_away = rolling_averages(df_away, cols, new_cols)
    df_home.index = range(0,df_home.shape[0])
    df_away.index = range(0,df_away.shape[0])
    return df_home, df_away

In [9]:
def df_with_target(team_id):
    df_home, df_away = add_new_cols(team_id)
    df_home["W"] = (df_home["WL_HOME"] == "W").astype("int")
    df_away["W"] = (df_away["WL_AWAY"] == "W").astype("int")
    df_home.index = range(0,df_home.shape[0])
    df_away.index = range(0,df_away.shape[0])
    return df_home, df_away

In [10]:
df1, df2 = df_with_target(knicks)
df1

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,PTS_AWAY_rolling,FGM_AWAY_rolling,FGA_AWAY_rolling,FG_PCT_AWAY_rolling,FG3M_AWAY_rolling,FG3A_AWAY_rolling,FG3_PCT_AWAY_rolling,FTM_AWAY_rolling,FTA_AWAY_rolling,FT_PCT_AWAY_rolling,OREB_AWAY_rolling,DREB_AWAY_rolling,REB_AWAY_rolling,AST_AWAY_rolling,TOV_AWAY_rolling,PLUS_MINUS_AWAY_rolling,PTS_HOME_rolling,FGM_HOME_rolling,FGA_HOME_rolling,FG_PCT_HOME_rolling,FG3M_HOME_rolling,FG3A_HOME_rolling,FG3_PCT_HOME_rolling,FTM_HOME_rolling,FTA_HOME_rolling,FT_PCT_HOME_rolling,OREB_HOME_rolling,DREB_HOME_rolling,REB_HOME_rolling,AST_HOME_rolling,TOV_HOME_rolling,PLUS_MINUS_HOME_rolling,W
0,22019,1610612758,SAC,Sacramento Kings,0021900087,2019-11-03,SAC @ NYK,W,241,113,41,85,0.482,15,39,0.385,16,25,0.640,10,40,50,25,7,5,14,29,21.0,1610612752,NYK,New York Knicks,NYK vs. SAC,L,241,92,32,80,0.400,8,31,0.258,20,31,0.645,9,29,38,18,6,5,13,23,-21.0,109.6,39.0,89.8,0.4360,14.0,38.4,0.3590,17.6,25.6,0.7048,11.2,34.8,46.0,22.0,14.2,7.4,102.2,35.4,85.8,0.4108,9.2,30.6,0.2980,22.2,31.0,0.7108,14.2,37.2,51.4,20.0,19.0,-7.4,0
1,22019,1610612739,CLE,Cleveland Cavaliers,0021900136,2019-11-10,CLE @ NYK,W,240,108,36,82,0.439,8,28,0.286,28,31,0.903,9,36,45,20,11,2,14,19,21.0,1610612752,NYK,New York Knicks,NYK vs. CLE,L,239,87,31,86,0.360,9,31,0.290,16,25,0.640,17,35,52,19,11,8,21,26,-21.0,109.2,39.2,89.0,0.4426,13.2,38.0,0.3434,17.6,26.0,0.6936,11.8,35.6,47.4,22.4,13.4,8.4,100.8,35.4,85.4,0.4128,8.8,30.0,0.2908,21.2,30.8,0.6836,13.6,35.0,48.6,19.4,17.0,-8.4,0
2,22019,1610612742,DAL,Dallas Mavericks,0021900162,2019-11-14,DAL @ NYK,L,239,103,36,84,0.429,8,36,0.222,23,31,0.742,12,32,44,21,8,7,11,16,-3.0,1610612752,NYK,New York Knicks,NYK vs. DAL,W,240,106,42,94,0.447,13,30,0.433,9,11,0.818,15,38,53,20,6,2,15,27,3.0,110.8,39.2,85.8,0.4570,12.8,36.8,0.3418,19.6,27.8,0.7106,10.4,34.4,44.8,21.8,14.0,11.8,99.0,35.2,86.4,0.4058,9.2,30.2,0.3022,19.4,29.2,0.6600,15.8,34.4,50.2,18.8,18.4,-11.8,1
3,22019,1610612766,CHA,Charlotte Hornets,0021900177,2019-11-16,CHA @ NYK,W,239,103,36,87,0.414,17,48,0.354,14,19,0.737,12,30,42,22,5,3,10,18,1.0,1610612752,NYK,New York Knicks,NYK vs. CHA,L,240,102,39,86,0.453,6,20,0.300,18,23,0.783,13,35,48,22,5,1,11,21,-1.0,108.0,38.2,85.8,0.4452,11.0,35.8,0.3032,20.6,27.4,0.7500,10.4,34.2,44.6,21.8,12.0,11.0,97.0,34.8,86.6,0.4006,9.2,29.6,0.3100,18.2,26.6,0.6986,16.4,35.8,52.2,18.2,17.8,-11.0,0
4,22019,1610612739,CLE,Cleveland Cavaliers,0021900189,2019-11-18,CLE @ NYK,L,240,105,36,76,0.474,17,36,0.472,16,30,0.533,10,31,41,18,2,1,19,26,-18.0,1610612752,NYK,New York Knicks,NYK vs. CLE,W,241,123,44,98,0.449,8,23,0.348,27,35,0.771,19,28,47,17,8,10,7,24,18.0,105.0,37.0,84.6,0.4376,11.4,37.2,0.3008,19.6,25.4,0.7664,10.2,33.6,43.8,21.4,11.6,6.6,98.4,36.4,87.4,0.4156,9.0,28.2,0.3182,16.6,24.4,0.7022,15.8,35.0,50.8,19.6,15.4,-6.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,22023,1610612758,SAC,Sacramento Kings,0022301112,2024-04-04,SAC @ NYK,L,241,109,42,86,0.488,18,42,0.429,7,12,0.583,9,26,35,26,6,3,16,23,-11.0,1610612752,NYK,New York Knicks,NYK vs. SAC,W,241,120,44,80,0.550,12,28,0.429,20,29,0.690,9,30,39,34,9,2,11,20,11.0,92.6,34.6,81.2,0.4266,10.4,33.0,0.3142,13.0,18.2,0.7162

In [11]:
#combines home games of all teams and away games of all teams
all_home, all_away = df_with_target(cavs)
    
for team in no_cavs:
    df_home, df_away = df_with_target(team)
    all_home = all_home.append(df_home)
    all_away = all_away.append(df_away)
    
all_home.index = range(0,all_home.shape[0])
all_away.index = range(0,all_away.shape[0])
all_home = all_home.dropna()
all_away = all_away.dropna()
        
all_home

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,PTS_AWAY_rolling,FGM_AWAY_rolling,FGA_AWAY_rolling,FG_PCT_AWAY_rolling,FG3M_AWAY_rolling,FG3A_AWAY_rolling,FG3_PCT_AWAY_rolling,FTM_AWAY_rolling,FTA_AWAY_rolling,FT_PCT_AWAY_rolling,OREB_AWAY_rolling,DREB_AWAY_rolling,REB_AWAY_rolling,AST_AWAY_rolling,TOV_AWAY_rolling,PLUS_MINUS_AWAY_rolling,PTS_HOME_rolling,FGM_HOME_rolling,FGA_HOME_rolling,FG_PCT_HOME_rolling,FG3M_HOME_rolling,FG3A_HOME_rolling,FG3_PCT_HOME_rolling,FTM_HOME_rolling,FTA_HOME_rolling,FT_PCT_HOME_rolling,OREB_HOME_rolling,DREB_HOME_rolling,REB_HOME_rolling,AST_HOME_rolling,TOV_HOME_rolling,PLUS_MINUS_HOME_rolling,W
0,22019,1610612738,BOS,Boston Celtics,0021900098,2019-11-05,BOS @ CLE,W,242,119,48,85,0.565,12,30,0.400,11,15,0.733,4,41,45,24,5,12,14,23,6.0,1610612739,CLE,Cleveland Cavaliers,CLE vs. BOS,L,238,113,40,95,0.421,13,33,0.394,20,22,0.909,11,32,43,22,3,6,9,18,-6.0,109.6,40.8,85.2,0.4796,13.8,34.8,0.3908,14.2,18.0,0.8348,8.0,31.6,39.6,24.0,16.0,-1.0,110.6,39.8,84.0,0.4756,12.6,36.0,0.3542,18.4,25.2,0.7384,9.8,31.4,41.2,23.6,15.2,1.0,0
1,22019,1610612748,MIA,Miami Heat,0021900161,2019-11-14,MIA @ CLE,W,239,108,43,79,0.544,12,30,0.400,10,11,0.909,4,37,41,31,10,7,18,21,11.0,1610612739,CLE,Cleveland Cavaliers,CLE vs. MIA,L,242,97,36,85,0.424,8,32,0.250,17,21,0.810,11,30,41,18,14,2,18,16,-11.0,115.6,43.4,86.2,0.5050,14.2,35.0,0.4018,14.6,19.2,0.7814,7.2,33.8,41.0,25.4,13.6,6.4,109.2,38.6,85.2,0.4564,12.6,35.0,0.3646,19.4,26.2,0.7438,10.0,31.4,41.4,22.6,14.2,-6.4,0
2,22019,1610612755,PHI,Philadelphia 76ers,0021900183,2019-11-17,PHI @ CLE,W,240,114,46,84,0.548,9,23,0.391,13,17,0.765,4,40,44,33,6,11,13,22,19.0,1610612739,CLE,Cleveland Cavaliers,CLE vs. PHI,L,240,95,34,90,0.378,6,25,0.240,21,24,0.875,11,31,42,14,9,5,14,21,-19.0,113.6,43.8,87.0,0.5044,13.6,33.6,0.4008,12.4,15.0,0.8320,6.8,34.2,41.0,27.4,13.6,4.0,109.6,39.2,86.8,0.4554,12.0,33.6,0.3582,19.2,25.0,0.7724,11.2,32.0,43.2,22.4,14.2,-4.0,0
3,22019,1610612757,POR,Portland Trail Blazers,0021900231,2019-11-23,POR @ CLE,L,240,104,39,87,0.448,14,40,0.350,12,16,0.750,6,32,38,21,6,6,10,20,-6.0,1610612739,CLE,Cleveland Cavaliers,CLE vs. POR,W,241,110,37,87,0.425,10,31,0.323,26,32,0.813,9,43,52,19,9,2,13,14,6.0,116.6,45.2,86.4,0.5244,13.4,32.6,0.4076,12.8,15.8,0.8158,6.4,36.2,42.6,29.4,12.8,10.0,106.6,38.0,88.8,0.4310,11.0,32.4,0.3352,19.6,24.0,0.8164,11.0,31.8,42.8,20.6,13.4,-10.0,1
4,22019,1610612751,BKN,Brooklyn Nets,0021900240,2019-11-25,BKN @ CLE,W,239,108,44,94,0.468,11,34,0.324,9,11,0.818,12,36,48,29,8,10,9,21,2.0,1610612739,CLE,Cleveland Cavaliers,CLE vs. BKN,L,239,106,40,94,0.426,13,32,0.406,13,20,0.650,12,33,45,24,5,4,10,14,-2.0,115.2,45.2,85.8,0.5274,13.4,32.8,0.4058,11.4,14.4,0.8006,5.8,38.0,43.8,28.6,13.6,10.0,105.2,37.2,90.6,0.4108,11.0,33.8,0.3164,19.8,24.2,0.8178,10.8,33.0,43.8,19.4,12.6,-10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6429,22023,1610612748,MIA,Miami Heat,0022301081,2024-03-31,MIA @ WAS,W,240,119,40,81,0.494,14,36,0.389,25,29,0.862,6,36,42,22,11,8,13,19,12.0,1610612764,WAS,Washington Wizards,WAS vs. MIA,L,240,107,40,88,0.455,11,38,0.289,16,21,0.762,7,35,42,27,6,1,15,22,-12.0,113.2,41.6,91.6,0.4536,9.6,33.6,0.2820,20.4,

In [16]:
model = GaussianNB()

home_train, home_test = train_test_split(all_home, test_size=0.15)
model_home = model.fit(home_train[pred_cols], home_train["W"])
preds = model.predict(home_test[pred_cols])
combined_home = pd.DataFrame(dict(actual=home_test["W"], predicted=preds), index=home_test.index)
accuracy_home = accuracy_score(home_test["W"], preds)
precision_home = precision_score(home_test["W"], preds)

away_train, away_test = train_test_split(all_away, test_size=0.15)
model_away = model.fit(away_train[pred_cols], away_train["W"])
preds = model.predict(away_test[pred_cols])
combined_away = pd.DataFrame(dict(actual=away_test["W"], predicted=preds), index=away_test.index)
accuracy_away = accuracy_score(away_test["W"], preds)
precision_away = precision_score(away_test["W"], preds)

print("Home")
print("Accuracy: " + str(accuracy_home))
print("Precision: " + str(precision_home))
print()
print("Away")
print("Accuracy: " + str(accuracy_away))
print("Precision: " + str(precision_away))

combined_away

Home
Accuracy: 0.950259067357513
Precision: 0.9514018691588785

Away
Accuracy: 0.9667359667359667
Precision: 0.9546485260770975


,actual,predicted
2849,0,0
1574,0,0
161,0,0
969,0,0
528,1,1
...,...,...
2881,1,1
6122,0,0
1036,0,0
4857,1,1


In [13]:
def predictor(away_id, home_id):
    df_home, df_away1 = df_with_target(home_id)
    df_home1, df_away = df_with_target(away_id)
    
    home_stats = list(df_home.loc[:, 'PTS_HOME_rolling':'PLUS_MINUS_HOME_rolling'].iloc[-1])
    home_pm = df_home['PLUS_MINUS_HOME'].iloc[-1]
    home_stats.append(home_pm)
    away_stats = list(df_away.loc[:, 'PTS_AWAY_rolling':'PLUS_MINUS_AWAY_rolling'].iloc[-1])
    away_pm = df_away['PLUS_MINUS_AWAY'].iloc[-1]
    away_stats.append(away_pm)
    new_stats = [away_stats+home_stats]
        
    df_predict = pd.DataFrame(new_stats, columns=pred_cols)
    df_predict['W'] = model_away.predict(new_stats)
    
    if df_predict['W'].iloc[0] == 1:
        result = "win"
    else: 
        result = "lose"
        
    print(f"The away team will {result} against the home team.")

In [115]:
# 4/26 - 2/3
predictor(bucks, pacers)
predictor(clippers, mavericks)
predictor(timberwolves, suns)

The away team will lose against the home team.
The away team will win against the home team.
The away team will win against the home team.


In [14]:
# 4/27 - x/4
predictor(cavs, magic)
predictor(thunder, pelicans)
predictor(celtics, heat)
predictor(nuggets, lakers)

The away team will lose against the home team.
The away team will lose against the home team.
The away team will lose against the home team.
The away team will win against the home team.


In [17]:
# 4/28 - x/4
predictor(knicks, sixers)
predictor(clippers, mavericks)
predictor(bucks, pacers)
predictor(timberwolves, suns)

The away team will lose against the home team.
The away team will lose against the home team.
The away team will lose against the home team.
The away team will win against the home team.
